In [1]:
from nltk import word_tokenize
import random

In [48]:
filename = "dataset/Alice in wonderland.txt"
sequenceLength = 1

In [49]:
wordList = []
tempMapping = {}
mapping = {}

In [50]:
with open(filename, 'r', encoding="utf8") as f:
    rawText = f.read().lower()
wordList = word_tokenize(rawText)
#print(wordList)

In [51]:
def addItemToMapping(history, word):
    while len(history) > 0:
        first = tuple(history)
        if first in tempMapping:
            if word in tempMapping[first]:
                tempMapping[first][word] = tempMapping[first][word] + 1.0
            else:
                tempMapping[first][word] = 1.0
        else:
            tempMapping[first] = {}
            tempMapping[first][word] = 1.0
        history = history[1:]

In [52]:
def build():
    for i in range(sequenceLength,len(wordList)-1):
        history = wordList[i-sequenceLength:i]
        follow = wordList[i+1]
        addItemToMapping(history, follow)
    #print(tempMapping)
            

In [53]:
build()

In [54]:
def normalize():
    for first, follow in iter(tempMapping.items()):
        total = sum(follow.values())
        mapping[first] = dict([(k,v/total) for k,v in iter(follow.items())])
        
    #print (mapping)

In [55]:
normalize()

In [56]:
def generateSentence():
    sent = ""
    lengthToGenerate = 20
    curr = random.choice(wordList)
    sent = sent + curr
    prevList = [curr]
    for i in range(1,lengthToGenerate):
        curr = next(prevList)
        prevList.append(curr)
        sent = sent + " " + curr
    
    return sent
    

In [57]:
def next(prevList):
    print(prevList)
    sum = 0.0
    retval = ""
    index = random.random()
    while tuple(prevList) not in mapping:
        prevList.pop(0)
    for k, v in iter(mapping[tuple(prevList)].items()):
        sum += v
        if sum >= index and retval == "":
            retval = k
    return retval

In [58]:
print(generateSentence())

['tone']
['tone', '‘']
['‘', 'and']
['and', 'bright']
['bright', 'eager']
['eager', 'many']
['many', 'to']
['to', '--']
['--', 'constant']
['constant', 'and']
['and', 'as']
['as', 'spoke']
['spoke', 'and']
['and', '--']
['--', 'know']
['know', 'any']
['any', '!']
['!', ',']
[',', 'and']
tone ‘ and bright eager many to -- constant and as spoke and -- know any ! , and ’
